In [7]:
# =============================================================================
# 0. PRÉPARATION DE L'ENVIRONNEMENT ET CHARGEMENT DES DONNÉES
# =============================================================================

# Cellule 0 : Imports, Configuration, Chargement & Fiche Synthétique (Parties 0, 1, 2 & 3.1-3.2)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

# Configuration pour l'affichage
pd.set_option('display.max_columns', None)
sns.set_style("whitegrid")

# Définir le chemin d'accès au fichier (Assurez-vous que le fichier est dans data/raw/)
# IMPORTANT : Le chemin est ajusté pour charger le fichier CSV nettoyé tel qu'indiqué par l'utilisateur.
# Si vous exécutez localement, utilisez le chemin complet : 'C:/Users/gabri/INGE4/Data vis/CLV-RFM-Marketing-App/data_clean.csv'
FILE_PATH = '../data_clean.csv' 

try:
    # Charger les données (format CSV)
    # Définir l'encodage 'latin1' ou 'ISO-8859-1' si vous rencontrez des problèmes d'accents
    df = pd.read_csv(FILE_PATH, encoding='utf-8') 
except FileNotFoundError:
    print(f"Erreur : Le fichier {FILE_PATH} n'a pas été trouvé. Veuillez vérifier le chemin ou le placer dans le dossier data/processed.")
    df = pd.DataFrame() # Créer un DataFrame vide pour éviter les erreurs de code suivantes

if not df.empty:
    # Renommer les colonnes pour la cohérence et l'utilisation future dans Streamlit.
    # Les noms sont basés sur la structure fournie par l'utilisateur, et TotalAmount est renommé 'sales'.
    df.columns = ['invoice_no', 'stock_code', 'description', 'quantity', 'invoice_date',
                  'price', 'customer_id', 'country', 'sales'] # 'TotalAmount' devient 'sales'

    # S'assurer que les colonnes 'invoice_date', 'customer_id', et 'sales' ont les bons types
    df['invoice_date'] = pd.to_datetime(df['invoice_date'])
    df['customer_id'] = df['customer_id'].astype(str)
    
    # Affichage de la Fiche Synthétique
    print("="*50)
    print("1. FICHE SYNTHÉTIQUE DES DONNÉES")
    print("="*50)
    print(f"Source : Online Retail II (UCI) - Version Nettoyée CSV")
    print(f"Période Couverte : {df['invoice_date'].min().date()} à {df['invoice_date'].max().date()}")
    print(f"Volume Initial : {len(df):,} lignes")
    print(f"Nombre de Clients Uniques (avant nettoyage) : {df['customer_id'].nunique():,}")
    print(f"Pays Couverts : {df['country'].nunique()} pays")
    
    # Affichage du Dictionnaire des Variables
    print("\n" + "="*50)
    print("2. DICTIONNAIRE DES VARIABLES")
    print("="*50)
    data_dict = pd.DataFrame({
        'Nom': df.columns,
        'Type Dtype': df.dtypes,
        'Sémantique': [
            'Identifiant de transaction/facture (commence par C pour annulation)',
            'Identifiant du produit',
            'Description du produit',
            'Quantité commandée',
            'Date et heure de la transaction',
            'Prix unitaire',
            'Identifiant unique du client',
            'Pays de destination',
            'Montant total de la ligne (Quantity * Price) - clé pour M du RFM et CLV'
        ]
    })
    print(data_dict)



1. FICHE SYNTHÉTIQUE DES DONNÉES
Source : Online Retail II (UCI) - Version Nettoyée CSV
Période Couverte : 2009-12-01 à 2010-12-09
Volume Initial : 400,916 lignes
Nombre de Clients Uniques (avant nettoyage) : 4,312
Pays Couverts : 37 pays

2. DICTIONNAIRE DES VARIABLES
                       Nom      Type Dtype  \
invoice_no      invoice_no           int64   
stock_code      stock_code          object   
description    description          object   
quantity          quantity           int64   
invoice_date  invoice_date  datetime64[ns]   
price                price         float64   
customer_id    customer_id          object   
country            country          object   
sales                sales         float64   

                                                     Sémantique  
invoice_no    Identifiant de transaction/facture (commence p...  
stock_code                               Identifiant du produit  
description                              Description du produit  
quant

In [8]:
# =============================================================================
# 3. QUALITÉ ET NETTOYAGE DES DONNÉES
# =============================================================================

# Cellule 1 : Qualité des Données (Manquants, Doublons, Outliers, Annulations)

if not df.empty:
    print("\n" + "="*50)
    print("3. QUALITÉ DES DONNÉES")
    print("="*50)

    # 3.1. Valeurs Manquantes
    print("\n3.1. Valeurs Manquantes (NaN) :")
    nan_info = df.isnull().sum()
    nan_percent = (df.isnull().sum() / len(df)) * 100
    print(pd.DataFrame({'Nombre': nan_info, 'Pourcentage': nan_percent.round(2)}).sort_values(by='Nombre', ascending=False))

    # NOTE : 'customer_id' a ~25% de valeurs manquantes. Ces lignes DOIVENT être retirées pour RFM/CLV.

    # 3.2. Doublons
    duplicates = df.duplicated().sum()
    print(f"\n3.2. Nombre de lignes dupliquées : {duplicates}")

    # 3.3. Outliers et Règles d'Annulation
    print("\n3.3. Aperçu des Outliers (Quantity/Price) et Annulations :")
    print(df[['quantity', 'price', 'sales']].describe().T)

    # 3.4. Création du DataFrame nettoyé (df_transactions)
    
    # 1. Suppression des NaN (principalement customer_id)
    df_transactions = df.dropna(subset=['customer_id']).copy()

    # 2. Suppression des doublons
    df_transactions.drop_duplicates(inplace=True)

    # 3. Traitement des transactions non-valides et des annulations
    
    # Séparer les annulations (InvoiceNo commençant par 'C')
    # Assurez-vous que la colonne 'invoice_no' est traitée comme une chaîne
    df_cancellations = df_transactions[df_transactions['invoice_no'].astype(str).str.startswith('C')]
    
    # Conserver uniquement les transactions valides (Quantity > 0 et Price > 0)
    # Exclure les lignes où la facture est une annulation et où la colonne 'sales' est positive
    df_transactions = df_transactions[
        (~df_transactions['invoice_no'].astype(str).str.startswith('C')) & 
        (df_transactions['quantity'] > 0) & 
        (df_transactions['price'] > 0) &
        (df_transactions['sales'] > 0) # La colonne 'sales' doit aussi être positive
    ].copy()
    
    print(f"\nVolume après nettoyage (lignes valides, clients identifiés) : {len(df_transactions):,} lignes")
    print(f"Volume des annulations identifiées : {len(df_cancellations):,} lignes")



3. QUALITÉ DES DONNÉES

3.1. Valeurs Manquantes (NaN) :
              Nombre  Pourcentage
invoice_no         0          0.0
stock_code         0          0.0
description        0          0.0
quantity           0          0.0
invoice_date       0          0.0
price              0          0.0
customer_id        0          0.0
country            0          0.0
sales              0          0.0

3.2. Nombre de lignes dupliquées : 0

3.3. Aperçu des Outliers (Quantity/Price) et Annulations :
             count       mean        std    min   25%    50%    75%      max
quantity  400916.0  13.767418  97.638385  1.000  2.00   5.00  12.00  19152.0
price     400916.0   3.305826  35.047719  0.001  1.25   1.95   3.75  10953.5
sales     400916.0  21.945330  77.758075  0.001  5.00  12.50  19.50  15818.4

Volume après nettoyage (lignes valides, clients identifiés) : 400,916 lignes
Volume des annulations identifiées : 0 lignes


In [2]:
# -*- coding: utf-8 -*-
# <nbformat>4.1</nbformat>

# <codecell>

import pandas as pd
import numpy as np

# Configuration pour afficher toutes les colonnes des DataFrames
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Définition du chemin du fichier de données
DATA_PATH = '../app/data/raw/online_retail_ii.xlsx' # Assurez-vous d'avoir le fichier dans ce chemin

# Chargement du jeu de données (Online Retail II)
try:
    # Lecture du fichier Excel. Assurez-vous d'utiliser le bon nom de fichier (ici .xlsx)
    df = pd.read_excel(DATA_PATH)
    print("Données chargées avec succès.")
except FileNotFoundError:
    print(f"ERREUR : Le fichier de données est introuvable à l'adresse {DATA_PATH}.")
    print("Veuillez télécharger le jeu de données 'Online Retail II' et le placer dans 'app/data/raw/'.")
    df = pd.DataFrame() # Crée un DataFrame vide en cas d'erreur
    
# Affichage des 5 premières lignes pour un aperçu rapide
if not df.empty:
    display(df.head())

# <markdown>
# # Partie I : Notebook d'Exploration Visuelle Complète
# 
# L'objectif de cette première phase est d'acquérir une compréhension exhaustive du jeu de données *Online Retail II* (UCI) afin de cadrer les analyses et la construction de l'application Streamlit.
# 
# ---
# 
# ## Fiche Synthétique des Données
# 
# ### 1. Source, Volume et Période

# <codecell>

if not df.empty:
    # 1. Volume et Période
    print(f"Source: Online Retail II (UCI)")
    print(f"Volume: {df.shape[0]:,} lignes et {df.shape[1]} colonnes")
    
    # Conversion de la colonne d'horodatage si nécessaire (assumant qu'elle est déjà en datetime si l'on utilise read_excel)
    # df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate']) 
    
    min_date = df['InvoiceDate'].min()
    max_date = df['InvoiceDate'].max()
    
    print(f"Période couverte : Du {min_date.strftime('%d/%m/%Y')} au {max_date.strftime('%d/%m/%Y')}")
    print(f"Clients uniques : {df['Customer ID'].nunique():,} (excluant les valeurs manquantes)")

# <markdown>
# ### 2. Dictionnaire des Variables (À Compléter)
# 
# Remplissez ce tableau avec les détails de chaque colonne.
# 
# | Nom de la Variable | Type (Python) | Sémantique | Unités/Valeurs | Colonne Importante? |
# |:------------------|:--------------|:-----------|:---------------|:-------------------|
# | `InvoiceNo`       | object        | Numéro de facture. Commence par 'C' si annulation (retour). | Entier/Chaîne de caractères | Oui |
# | `StockCode`       | object        | Code produit (Item). | Chaîne de caractères | Oui |
# | `Description`     | object        | Nom du produit. | Chaîne de caractères | Oui |
# | `Quantity`        | int64         | Quantité de produits achetés par transaction. | Nombre entier | Oui |
# | `InvoiceDate`     | datetime64[ns]| Date et heure de la transaction. | Horodatage | Oui |
# | `Price`           | float64       | Prix unitaire du produit. | Monétaire (GBP, assumé) | Oui |
# | `Customer ID`     | float64       | Identifiant unique du client. | Entier (float avec NaN) | Oui |
# | `Country`         | object        | Pays de destination de la commande. | Chaîne de caractères | Oui |
# 
# ## Qualité des Données
# 
# ### 3. Valeurs Manquantes

# <codecell>

if not df.empty:
    print("Aperçu des valeurs manquantes par colonne (en %):")
    missing_data = df.isnull().sum() / len(df) * 100
    display(missing_data[missing_data > 0].sort_values(ascending=False))
    
    # Notez l'importance des ID clients manquants pour l'analyse CLV/Cohortes
    nan_customers_count = df['Customer ID'].isnull().sum()
    print(f"\nTotal des lignes sans Customer ID : {nan_customers_count:,} ({missing_data['Customer ID']:.2f} % des données).")

# <markdown>
# ### 4. Règles d'Annulation (Retours) et Outliers
# 
# **Règle :** Les factures (`InvoiceNo`) commençant par "C" sont des annulations ou des retours.

# <codecell>

if not df.empty:
    # Identifier les retours
    df['is_return'] = df['InvoiceNo'].astype(str).str.startswith('C')
    return_count = df['is_return'].sum()
    
    print(f"Nombre total de transactions : {df.shape[0]:,}")
    print(f"Nombre de retours (lignes 'C...') : {return_count:,} ({return_count/df.shape[0]*100:.2f} % du total)")

    # Vérification des outliers sur Quantity et Price
    print("\nStatistiques sur les quantités et les prix (incluant les retours/annulations):")
    display(df[['Quantity', 'Price']].describe().T)

    # Filtrer les quantités ou prix nuls/négatifs (autres que les retours marqués 'C')
    negative_quantity = df[df['Quantity'] < 0].shape[0]
    zero_price = df[df['Price'] <= 0].shape[0]

    print(f"\nLignes avec Quantity < 0 (hors 'C'): {negative_quantity:,}")
    print(f"Lignes avec Price <= 0: {zero_price:,}")
    
    print("\n--- Poursuivre avec la suppression des doublons et la création des premières métriques (ex: TotalPrice) ---")

# <markdown>
# ## Prochaine Étape : Nettoyage et Création de Métriques
# 
# Une fois cette exploration terminée, l'étape logique est :
# 1. **Nettoyer** : Supprimer les lignes avec ID client manquant.
# 2. **Prétraiter** : Gérer les retours (les neutraliser, les exclure, ou les utiliser, selon l'approche souhaitée).
# 3. **Calculer** : Créer la colonne `TotalRevenue` (`Quantity` * `Price`).
# 4. **Visualiser** : Créer les 6-8 graphiques demandés.

# <codecell>
# Fin du Notebook initial.

ERREUR : Le fichier de données est introuvable à l'adresse ../app/data/raw/online_retail_ii.xlsx.
Veuillez télécharger le jeu de données 'Online Retail II' et le placer dans 'app/data/raw/'.
